References:
- https://docs.python.org/3/reference/datamodel.html
- Fluent Python by Luciano Ramalho. Chapter 16: Coroutines

# Coroutines for Discrete Event Simulation


In [1]:
import sys
import random
import collections
import queue

In [2]:
DEFAULT_NUMBER_OF_TAXIS = 3
DEFAULT_END_TIME = 180
SEARCH_DURATION = 5
TRIP_DURATION = 20
DEPARTURE_INTERVAL = 5

#### `Event` instance
 * `time` is the simulation time
 * `proc` is the process instance
 * `action` is a string describing the event

In [3]:
Event = collections.namedtuple('Event', 'time proc action')

In [4]:
# Returns time interval in minutes
def compute_delay(interval):
    """Compute action delay using exponential distribution"""
    return int(random.expovariate(1/interval)) + 1

* `taxi_process` will be called once per taxi, creating a generatorobject to represent its operations. 
    * `ident` is the index number of the taxi (eg. 0, 1, 2 in the sample run);  
    * `trips` is the number of trips this taxi will make before going home;  
    * `start_time` is when the taxi leaves the garage.

In [5]:
def taxi_process(ident, trips, start_time=0):  # <1>
    """Yield to simulator issuing event at each state change"""
    time = yield Event(start_time, ident, 'leave garage')  # <2>
    for i in range(trips):  # repeated once for each trip
        prowling_ends = time + compute_delay(SEARCH_DURATION)  # The ending time of the search  for a passenger.
        time = yield Event(prowling_ends, ident, 'pick up passenger')  # <5>

        trip_ends = time + compute_delay(TRIP_DURATION)  # ending time of trip
        time = yield Event(trip_ends, ident, 'drop off passenger')  # <7>

    yield Event(time + 1, ident, 'going home')  # <8>
    # end of taxi process # raises `StopIeteration`

* <2> The first `Event` yielded is `'leave garage'`. This suspends the coroutine, and lets the simulation main loop proceed to the next scheduled event. When it's time to reactivate this process, the main loop will `send` the current simulation time, which is assigned to `time`.


* <5> An `Event` signaling passenger pick up is yielded. The coroutine pauses here. When the time comes to reactivate this coroutine, the main loop will again `send` the current time.


* <7> An `Event` signaling passenger drop off is yielded. Coroutine suspended again, waiting for the main loop to send the time of when it's time to continue.


* <8> The `for` loop ends after the given number of trips, and a final `'going home'` event is yielded, to happen 1 minute after the current time. The coroutine will suspend for the last time. When reactivated, it will be sent the time from the simulation main loop, but here I don't assign it to any variable because it will not be useful.

In [6]:
# Create a generator object
taxi = taxi_process(ident=13, trips=2, start_time=0)

In [7]:
# Prime the coroutine
next(taxi)

Event(time=0, proc=13, action='leave garage')

In [8]:
# Taxi will spend 7 minutes searching for first passenger
taxi.send(_.time + 7)

# output yielded by  the for loop at start of trip

Event(time=12, proc=13, action='pick up passenger')

In [9]:
# The trip with the first passenger to be 23 minutes
taxi.send(_.time + 23)

Event(time=46, proc=13, action='drop off passenger')

In [10]:
# Then the taxi will prowl for 5 minutes
taxi.send(_.time + 5)

Event(time=55, proc=13, action='pick up passenger')

In [11]:
# The last trip will take 48 minutes
taxi.send(_.time + 48)

Event(time=129, proc=13, action='drop off passenger')

In [12]:
# The loop will endd after 2 trips
taxi.send(_.time + 1)

Event(time=131, proc=13, action='going home')

In [13]:
# The next attempt will fail
taxi.send(_.time + 10)

StopIteration: 

##  Taxi Simulator

In [14]:
class Simulator:

    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)


    def run(self, end_time):
        """Schedule and display events until time is up"""
        # schedule the first event for each cab
        for _, proc in sorted(self.procs.items()):  # retrieve the self.procs items ordered by integer key
            first_event = next(proc)  # primes each coroutine
            self.events.put(first_event)  #add each event

        # main loop of the simulation
        time = 0
        while time < end_time:  # main loop
            if self.events.empty():  # main loop may exit if there is no pending event
                print('*** end of events ***')
                break

            # get and display current event
            current_event = self.events.get()  # get event with the smallest time
            print('taxi:', current_event.proc,  # display the event 
                  current_event.proc * '   ', current_event)

            # schedule next action for current proc
            time = current_event.time
            proc = self.procs[current_event.proc]  # retrieve the coroutine
            try:
                next_event = proc.send(time)  # send the time to the taxi coroutine, which will yield next event
            except StopIteration:
                del self.procs[current_event.proc]  # delete the coroutine from the procs dict
            else:
                self.events.put(next_event)  # otherwise put event in queue
        else:  # on exit main loop
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))

In [15]:
num_taxis = DEFAULT_NUMBER_OF_TAXIS

taxis = {i: taxi_process(i, (i+1)*2, i*DEPARTURE_INTERVAL)
         for i in range(num_taxis)}

sim = Simulator(taxis)

In [16]:
end_time = DEFAULT_END_TIME

sim.run(end_time)

taxi: 0  Event(time=0, proc=0, action='leave garage')
taxi: 1     Event(time=5, proc=1, action='leave garage')
taxi: 0  Event(time=6, proc=0, action='pick up passenger')
taxi: 1     Event(time=10, proc=1, action='pick up passenger')
taxi: 2        Event(time=10, proc=2, action='leave garage')
taxi: 1     Event(time=14, proc=1, action='drop off passenger')
taxi: 1     Event(time=17, proc=1, action='pick up passenger')
taxi: 2        Event(time=17, proc=2, action='pick up passenger')
taxi: 0  Event(time=18, proc=0, action='drop off passenger')
taxi: 0  Event(time=19, proc=0, action='pick up passenger')
taxi: 1     Event(time=31, proc=1, action='drop off passenger')
taxi: 2        Event(time=31, proc=2, action='drop off passenger')
taxi: 2        Event(time=40, proc=2, action='pick up passenger')
taxi: 1     Event(time=43, proc=1, action='pick up passenger')
taxi: 1     Event(time=55, proc=1, action='drop off passenger')
taxi: 0  Event(time=58, proc=0, action='drop off passenger')
taxi: 0